In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [26]:
train = pd.read_csv('train.csv',index_col= 'PassengerId')
train['Age'].dtype

dtype('float64')

In [27]:
train.shape
test = pd.read_csv('test.csv',index_col= 'PassengerId')
test.head()


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [28]:
train.isnull().sum()
test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

In [30]:
#import missingno as mn
#mn.matrix(train)

In [31]:
# Data Preprocessing
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan , strategy='median')
Age_2 = train['Age'].values.reshape(1,-1)
Age_2 = Age_2.T
Age_2 = imputer.fit_transform(Age_2)
train['Age_2'] = Age_2
train.head()

Age_test = test['Age'].values.reshape(1,-1)
Age_test = Age_test.T
Age_test = imputer.fit_transform(Age_test)
test['Age_2'] = Age_test
test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_2
PassengerId,,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,34.5
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,47.0
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,62.0
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,27.0
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,22.0


In [32]:
train.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
Age_2         0
dtype: int64

In [33]:
# Taking care of the embarked column
# lets see the values in Embarked column

train['Embarked'].value_counts() # As we see that 'S' value is very frequent, we replace the null with 'S'
train.Embarked.fillna('S',inplace = True)

In [34]:
#Any other missing value in test data set
test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
Age_2         0
dtype: int64

In [35]:
test['Fare'].fillna(test['Fare'].mean(),inplace = True)
#test.isnull().sum()

In [36]:
# We drop cabin column as it has nearly 80% in NA
print(train['Cabin'].isnull().sum()/len(train['Cabin']))
test['Cabin'].isnull().sum()/len(test['Cabin'])

0.7710437710437711


0.7822966507177034

In [37]:
train.drop(['Age','Cabin'], inplace = True, axis = 1)
test.drop(['Age','Cabin'], inplace = True, axis = 1)

In [38]:
train.head()

,Survived,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Embarked,Age_2
PassengerId,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,1,0,A/5 21171,7.2500,S,22.0
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,1,0,PC 17599,71.2833,C,38.0
3,1,3,"Heikkinen, Miss. Laina",female,0,0,STON/O2. 3101282,7.9250,S,26.0
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,1,0,113803,53.1000,S,35.0
5,0,3,"Allen, Mr. William Henry",male,0,0,373450,8.0500,S,35.0


In [39]:
test.head() 

,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Embarked,Age_2
PassengerId,,,,,,,,,
892,3,"Kelly, Mr. James",male,0,0,330911,7.8292,Q,34.5
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,1,0,363272,7.0000,S,47.0
894,2,"Myles, Mr. Thomas Francis",male,0,0,240276,9.6875,Q,62.0
895,3,"Wirz, Mr. Albert",male,0,0,315154,8.6625,S,27.0
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,1,1,3101298,12.2875,S,22.0


In [40]:
# Remoing outliers

Q1 = train['Fare'].quantile(0.25)
Q3 = train['Fare'].quantile(0.75)

IQR = Q3-Q1
print(IQR)

train[(train['Fare'] < (Q1 - 1.5 * IQR)) | (train['Fare'] > (Q3 + 1.5 * IQR))]['Fare'].max()
train['Fare'].median

# We are not deleting any outliers as most is dependent on catagorial data and we dont ga=have any catagorical outliers

23.0896


<bound method Series.median of PassengerId
1       7.2500
2      71.2833
3       7.9250
4      53.1000
5       8.0500
        ...   
887    13.0000
888    30.0000
889    23.4500
890    30.0000
891     7.7500
Name: Fare, Length: 891, dtype: float64>

In [41]:
# we see that there is an outlier in fare that has to be removed
# so we dont consider any fare abover 400
train['Fare'].isnull().sum()
test.isnull().sum()

Pclass      0
Name        0
Sex         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Embarked    0
Age_2       0
dtype: int64

In [42]:
train = pd.get_dummies(train,columns  =['Sex','Embarked','Pclass'], drop_first = True)
test = pd.get_dummies(test,columns  =['Sex','Embarked','Pclass'], drop_first = True)

In [43]:
test.head()

,Name,SibSp,Parch,Ticket,Fare,Age_2,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
PassengerId,,,,,,,,,,,
892,"Kelly, Mr. James",0,0,330911,7.8292,34.5,1,1,0,0,1
893,"Wilkes, Mrs. James (Ellen Needs)",1,0,363272,7.0000,47.0,0,0,1,0,1
894,"Myles, Mr. Thomas Francis",0,0,240276,9.6875,62.0,1,1,0,1,0
895,"Wirz, Mr. Albert",0,0,315154,8.6625,27.0,1,0,1,0,1
896,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,1,3101298,12.2875,22.0,0,0,1,0,1


In [2]:
# Converting DataFrames to arrays for ML Algorithms
X_train = train.iloc[:,6:].values
y_train = train.iloc[:,0].values
x_test = test.iloc[:,5:].values

#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
x_test = sc.transform(x_test)
X_train

NameError: name 'train' is not defined

In [50]:
from sklearn.model_selection import train_test_split
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_train, y_train, test_size = 0.25, random_state = 1)

from sklearn.ensemble import RandomForestClassifier
classifierRF = RandomForestClassifier(criterion = 'entropy')

In [52]:
from sklearn.model_selection import RandomizedSearchCV

#Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 5, num = 5)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

#Cross validation is not required specially, as there is a parameter for CV in this itself

rf_random = RandomizedSearchCV(estimator = classifierRF, param_distributions = random_grid, n_iter = 1 00, cv = 5, verbose=2, n_jobs = -1)
rf_random.fit(X_train_1,y_train_1)
rf_random.best_params_,rf_random.best_score_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   15.0s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.5min finished


({'n_estimators': 500,
  'min_samples_split': 5,
  'min_samples_leaf': 1,
  'max_features': 'auto',
  'max_depth': 4,
  'bootstrap': False},
 0.8398173958712075)

In [58]:
classifierRF_1 = RandomForestClassifier(n_estimators = 500, max_depth = 4, min_samples_split=5,min_samples_leaf=1, max_features='auto', bootstrap= True)
classifierRF_1.fit(X_train_1,y_train_1)
y_pred = classifierRF_1.predict(X_test_1)


from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

cm = confusion_matrix(y_test_1, y_pred)
ac = accuracy_score(y_test_1, y_pred)
print(ac)

0.7802690582959642


In [40]:
y_test = classifierSVMR.predict(x_test)
test.reset_index(level=0, inplace=True)

In [41]:
y_test_2 = pd.Series(y_test)
Final = pd.DataFrame(test['PassengerId'])
Final['Survived'] = y_test

In [42]:
Final.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [43]:
Final.to_csv('Titanic_with SVMR_2.csv', index=False)

In [200]:
test

,PassengerId,Name,SibSp,Parch,Ticket,Fare,Age_2,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
0,892,"Kelly, Mr. James",0,0,330911,7.8292,34.5,1,1,0,0,1
1,893,"Wilkes, Mrs. James (Ellen Needs)",1,0,363272,7.0000,47.0,0,0,1,0,1
2,894,"Myles, Mr. Thomas Francis",0,0,240276,9.6875,62.0,1,1,0,1,0
3,895,"Wirz, Mr. Albert",0,0,315154,8.6625,27.0,1,0,1,0,1
4,896,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,1,3101298,12.2875,22.0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,"Spector, Mr. Woolf",0,0,A.5. 3236,8.0500,27.0,1,0,1,0,1
414,1306,"Oliva y Ocana, Dona. Fermina",0,0,PC 17758,108.9000,39.0,0,0,0,0,0
415,1307,"Saether, Mr. Simon Sivertsen",0,0,SOTON/O.Q. 3101262,7.2500,38.5,1,0,1,0,1
416,1308,"Ware, Mr. Frederick",0,0,359309,8.0500,27.0,1,0,1,0,1


In [59]:
import tensorflow

ModuleNotFoundError: No module named 'tensorflow'

In [60]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'